<a href="https://colab.research.google.com/github/Hassan-293/My-Project/blob/main/Unet_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import random
import numpy as np
from tqdm import tqdm
from skimage.io import imread,imshow
from skimage.transform import resize
import matplotlib.pyplot as plt

In [ ]:
train_image_dir = r'/content/drive/MyDrive/Histo Data/Train_Processed'
train_mask_dir = r'/content/drive/MyDrive/Histo Data/Labels_Processesd'

In [ ]:
train_image_paths = sorted([os.path.join(train_image_dir, fname) for fname in os.listdir(train_image_dir) if fname.endswith(".png") and not fname.startswith(".")])
train_mask_paths = sorted([os.path.join(train_mask_dir, fname) for fname in os.listdir(train_mask_dir) if fname.endswith(".png") and not fname.startswith(".")])
print("Number of training images : ", len(train_image_paths))
print("Number of training masks : ", len(train_mask_paths))

Number of training images :  238
Number of training masks :  238


In [ ]:
# Shuffle
import random
combined = list(zip(train_image_paths, train_mask_paths))
random.shuffle(combined)
train_image_paths[:], train_mask_paths[:] = zip(*combined)

In [ ]:
print("After shuffling")
print(train_image_paths[0:5])
print(train_mask_paths[0:5])
# Sequence intact !

After shuffling
['/content/drive/MyDrive/Histo Data/Train_Processed/crag_51.png', '/content/drive/MyDrive/Histo Data/Train_Processed/glas_5.png', '/content/drive/MyDrive/Histo Data/Train_Processed/dpath_23.png', '/content/drive/MyDrive/Histo Data/Train_Processed/crag_42.png', '/content/drive/MyDrive/Histo Data/Train_Processed/pannuke_25.png']
['/content/drive/MyDrive/Histo Data/Labels_Processesd/crag_51.png', '/content/drive/MyDrive/Histo Data/Labels_Processesd/glas_5.png', '/content/drive/MyDrive/Histo Data/Labels_Processesd/dpath_23.png', '/content/drive/MyDrive/Histo Data/Labels_Processesd/crag_42.png', '/content/drive/MyDrive/Histo Data/Labels_Processesd/pannuke_25.png']


In [ ]:
 #Splitting
train_image_files = train_image_paths[:190]
train_mask_files = train_mask_paths[:190]

valid_image_files = train_image_paths[:190]
valid_mask_files = train_mask_paths[:190]

print(len(train_image_files), len(train_mask_files))
print(len(valid_image_files), len(valid_mask_files))


190 190
190 190


In [ ]:

import tensorflow as tf

def unet_model(_imgHeight,  _imgWidth, _imgChannel):
    # Build the model:
    # inputs:
    inputs = tf.keras.layers.Input((_imgHeight,  _imgWidth, _imgChannel))
    s = tf.keras.layers.Lambda(lambda x : x / 255.0)(inputs)

    # Contraction path
    # First Layer:  
    C1 = tf.keras.layers.Conv2D(16, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
    C1 = tf.keras.layers.Dropout(0.1)(C1)
    C1 = tf.keras.layers.Conv2D(16, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(C1)
    P1 = tf.keras.layers.MaxPooling2D((2,2))(C1)
    # Second Layer:
    C2 = tf.keras.layers.Conv2D(32, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(P1)
    C2 = tf.keras.layers.Dropout(0.1)(C2)
    C2 = tf.keras.layers.Conv2D(32, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(C2)
    P2 = tf.keras.layers.MaxPooling2D((2,2))(C2)
    # Third Layer:
    C3 = tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(P2)
    C3 = tf.keras.layers.Dropout(0.2)(C3)
    C3 = tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(C3)
    P3 = tf.keras.layers.MaxPooling2D((2,2))(C3)
    # Forth Layer:
    C4 = tf.keras.layers.Conv2D(128, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(P3)
    C4 = tf.keras.layers.Dropout(0.2)(C4)
    C4 = tf.keras.layers.Conv2D(128, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(C4)
    P4 = tf.keras.layers.MaxPooling2D((2,2))(C4)
    # Fifth Layer:
    C5 = tf.keras.layers.Conv2D(256, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(P4)
    C5 = tf.keras.layers.Dropout(0.3)(C5)
    C5 = tf.keras.layers.Conv2D(256, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(C5)

    # Expansive Path:
    # Sixth Layer
    U6 = tf.keras.layers.Conv2DTranspose(128, (2,2), strides=(2,2), padding='same')(C5)
    U6 = tf.keras.layers.concatenate([U6, C4])
    C6 = tf.keras.layers.Conv2D(128, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(U6)
    C6 = tf.keras.layers.Dropout(0.2)(C6)
    C6 = tf.keras.layers.Conv2D(128, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(C6)
    # Seventh Layer:
    U7 = tf.keras.layers.Conv2DTranspose(64, (2,2), strides=(2,2), padding='same')(C6)
    U7 = tf.keras.layers.concatenate([U7, C3])
    C7 = tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(U7)
    C7 = tf.keras.layers.Dropout(0.2)(C7)
    C7 = tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(C7)
    # Eighth Layer:
    U8 = tf.keras.layers.Conv2DTranspose(32, (2,2), strides=(2,2), padding='same')(C7)
    U8 = tf.keras.layers.concatenate([U8, C2])
    C8 = tf.keras.layers.Conv2D(32, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(U8)
    C8 = tf.keras.layers.Dropout(0.1)(C8)
    C8 = tf.keras.layers.Conv2D(32, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(C8)
    # Ninth Layer:
    U9 = tf.keras.layers.Conv2DTranspose(16, (2,2), strides=(2,2), padding='same')(C8)
    U9 = tf.keras.layers.concatenate([U9, C1], axis=3)
    C9 = tf.keras.layers.Conv2D(16, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(U9)
    C9 = tf.keras.layers.Dropout(0.1)(C9)
    C9 = tf.keras.layers.Conv2D(16, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(C9)

    # Output Layer:
    outputs = tf.keras.layers.Conv2D(1, (1,1), activation='sigmoid')(C9)

    model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    model.summary()
    return model